In [1]:
import pandas as pd
import numpy as np
from scipy import c_, stats
from sklearn import metrics

import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("BankChurners.csv")

In [3]:
df = df.drop(['CLIENTNUM', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1)

In [4]:
df.drop(df[df['Marital_Status'] == "Unknown"].index, inplace=True)
df.drop(df[df['Income_Category'] == "Unknown"].index, inplace=True)
df.drop(df[df['Education_Level'] == "Unknown"].index, inplace=True)
df.drop(df[df['Card_Category'] == "Unknown"].index, inplace=True)

In [5]:
df['Attrition_Flag'] = df['Attrition_Flag'].replace(['Existing Customer'],'1')
df['Attrition_Flag'] = df['Attrition_Flag'].replace(['Attrited Customer'],'0')

In [6]:
df

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,1,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,1,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,1,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
4,1,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
5,1,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10120,1,54,M,1,High School,Single,$60K - $80K,Blue,34,5,2,0,13940.0,2109,11831.0,0.660,15577,114,0.754,0.151
10121,1,56,F,1,Graduate,Single,Less than $40K,Blue,50,4,1,4,3688.0,606,3082.0,0.570,14596,120,0.791,0.164
10122,1,50,M,2,Graduate,Single,$40K - $60K,Blue,40,3,2,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462
10124,0,44,F,1,High School,Married,Less than $40K,Blue,36,5,3,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000


In [7]:
X = pd.get_dummies(df, columns=['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category'], drop_first=True)
X = X.drop(['Attrition_Flag'],axis=1)

In [8]:
X_logreg = sm.add_constant(X)

/home/adam_s/um/moje_srod/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [9]:
X.columns

Index(['Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Gender_M', 'Education_Level_Doctorate', 'Education_Level_Graduate',
       'Education_Level_High School', 'Education_Level_Post-Graduate',
       'Education_Level_Uneducated', 'Marital_Status_Married',
       'Marital_Status_Single', 'Income_Category_$40K - $60K',
       'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K',
       'Income_Category_Less than $40K', 'Card_Category_Gold',
       'Card_Category_Platinum', 'Card_Category_Silver'],
      dtype='object')

In [10]:
X_logreg.describe()

,const,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,...,Education_Level_Uneducated,Marital_Status_Married,Marital_Status_Single,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
count,7081.0,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,...,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.00000,7081.000000,7081.000000,7081.000000,7081.000000
mean,1.0,46.347691,2.337805,35.981359,3.819376,2.342607,2.454456,8492.773831,1167.501624,7325.272207,...,0.170032,0.503319,0.416325,0.199407,0.155769,0.16975,0.394295,0.011439,0.001553,0.055218
std,0.0,8.041225,1.291649,8.002609,1.544444,0.995104,1.104917,9126.072520,812.315606,9131.217585,...,0.375688,0.500024,0.492984,0.399583,0.362662,0.37544,0.488733,0.106348,0.039386,0.228422
min,1.0,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,1.0,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2498.000000,463.000000,1248.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,1.0,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4287.000000,1282.000000,3250.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,1.0,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,10729.000000,1781.000000,9491.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000
max,1.0,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [11]:
y = df['Attrition_Flag']

In [12]:
y

0        1
1        1
2        1
4        1
5        1
        ..
10120    1
10121    1
10122    1
10124    0
10126    0
Name: Attrition_Flag, Length: 7081, dtype: object

In [13]:
X_train_logreg, X_test_logreg, y_train_logreg, y_test_logreg = train_test_split(X_logreg, y, random_state=42)


#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [108]:
y_train_logreg.astype(float).describe()

count    5310.000000
mean        0.841431
std         0.365308
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Attrition_Flag, dtype: float64

In [14]:
X_train_logreg.shape

(5310, 30)

In [15]:
y_train_logreg.shape

(5310,)

In [16]:
X_1 = X_train_logreg[[
    'Customer_Age',
    'Dependent_count',
    'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    #'Credit_Limit', 
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    'Avg_Utilization_Ratio',
    'Gender_M',
    'Education_Level_High School',
    'Education_Level_Graduate',
    'Education_Level_Uneducated',
    #'Education_Level_College',
    'Education_Level_Post-Graduate',
    'Education_Level_Doctorate',
    'Marital_Status_Married',
    'Marital_Status_Single',
    #'Marital_Status_Divorced',
    'Income_Category_$60K - $80K',
    'Income_Category_Less than $40K',
    'Income_Category_$80K - $120K',
    'Income_Category_$40K - $60K',
    #'Income_Category_$120K +',
    'Card_Category_Gold',
    'Card_Category_Silver',
    'Card_Category_Platinum'
               ]]
X_1 = sm.add_constant(X_1)

/home/adam_s/um/moje_srod/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [17]:
X_1

,const,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,...,Education_Level_Doctorate,Marital_Status_Married,Marital_Status_Single,Income_Category_$60K - $80K,Income_Category_Less than $40K,Income_Category_$80K - $120K,Income_Category_$40K - $60K,Card_Category_Gold,Card_Category_Silver,Card_Category_Platinum
8101,1.0,46,4,36,2,1,1,1043,1128.0,0.700,...,0,1,0,0,0,1,0,0,0,0
8916,1.0,51,1,37,1,3,2,2416,24902.0,0.917,...,0,0,1,0,0,1,0,0,0,0
7945,1.0,45,4,36,1,3,3,0,4688.0,0.593,...,0,0,1,0,0,0,1,0,0,0
10061,1.0,44,2,31,3,2,3,2028,2154.0,0.658,...,0,0,1,0,1,0,0,0,0,0
4743,1.0,48,3,42,3,2,4,0,19075.0,0.807,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,1.0,49,3,36,5,3,4,0,1438.3,0.929,...,0,1,0,0,1,0,0,0,0,0
7365,1.0,44,5,30,3,3,1,1726,983.0,0.698,...,0,0,1,0,1,0,0,0,0,0
7413,1.0,50,1,36,4,3,2,0,2317.0,0.734,...,0,0,1,1,0,0,0,0,0,0
7681,1.0,49,4,35,5,3,2,2133,905.0,0.689,...,0,1,0,0,1,0,0,0,0,0


In [31]:
log_reg_1 = sm.Logit(y_train_logreg.astype(float), X_1.astype(float)).fit()

Optimization terminated successfully.
         Current function value: 0.237683
         Iterations 8


In [32]:
print(log_reg_1.summary())

                           Logit Regression Results                           
Dep. Variable:         Attrition_Flag   No. Observations:                 5310
Model:                          Logit   Df Residuals:                     5281
Method:                           MLE   Df Model:                           28
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.4565
Time:                        16:14:15   Log-Likelihood:                -1262.1
converged:                       True   LL-Null:                       -2322.0
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -7.2959      0.631    -11.567      0.000      -8.532      -6.060
Customer_Age                       0.0134      0.011      1.259      0.

In [33]:
# test łącznej istotności Education_Level

x_vars = log_reg_1.summary2().tables[1].index
#x_vars
wald_str = ' = '.join(list(x_vars[15:20]))
wald_str

'Education_Level_High School = Education_Level_Graduate = Education_Level_Uneducated = Education_Level_Post-Graduate = Education_Level_Doctorate'

In [59]:
wald_test = log_reg_1.wald_test(wald_str) # joint test
wald_test

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[12.59149448]], p-value=0.005608707189585004, df_denom=3>

In [41]:
X_2 = X_train_logreg[[
    'Customer_Age',
    'Dependent_count',
    'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    #'Credit_Limit', 
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    'Avg_Utilization_Ratio',
    'Gender_M',
    #'Education_Level_High School',
    #'Education_Level_Graduate',
    #'Education_Level_Uneducated',
    #'Education_Level_College',
    #'Education_Level_Post-Graduate',
    #'Education_Level_Doctorate',
    'Marital_Status_Married',
    'Marital_Status_Single',
    #'Marital_Status_Divorced',
    'Income_Category_$60K - $80K',
    'Income_Category_Less than $40K',
    'Income_Category_$80K - $120K',
    'Income_Category_$40K - $60K',
    #'Income_Category_$120K +',
    'Card_Category_Gold',
    'Card_Category_Silver',
    'Card_Category_Platinum'
               ]]
X_2 = sm.add_constant(X_2)

/home/adam_s/um/moje_srod/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


**Bez edukacji**

In [42]:
log_reg_2 = sm.Logit(y_train_logreg.astype(float), X_2.astype(float)).fit()

Optimization terminated successfully.
         Current function value: 0.238503
         Iterations 8


In [43]:
print(log_reg_2.summary())

                           Logit Regression Results                           
Dep. Variable:         Attrition_Flag   No. Observations:                 5310
Model:                          Logit   Df Residuals:                     5286
Method:                           MLE   Df Model:                           23
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.4546
Time:                        16:15:05   Log-Likelihood:                -1266.5
converged:                       True   LL-Null:                       -2322.0
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -7.2979      0.615    -11.865      0.000      -8.503      -6.092
Customer_Age                       0.0131      0.011      1.245      0.

In [47]:
x_vars = log_reg_2.summary2().tables[1].index
#x_vars
wald_str = ' = '.join(list(x_vars[15:17]))
wald_str


'Marital_Status_Married = Marital_Status_Single'

In [58]:
wald_test = log_reg_2.wald_test(wald_str) # joint test
wald_test

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[12.36623003]], p-value=0.006228335136686634, df_denom=3>

**Bez Months_on_book**

In [51]:
X_3 = X_train_logreg[[
    'Customer_Age',
    'Dependent_count',
    #'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    #'Credit_Limit', 
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    'Avg_Utilization_Ratio',
    'Gender_M',
    #'Education_Level_High School',
    #'Education_Level_Graduate',
    #'Education_Level_Uneducated',
    #'Education_Level_College',
    #'Education_Level_Post-Graduate',
    #'Education_Level_Doctorate',
    'Marital_Status_Married',
    'Marital_Status_Single',
    #'Marital_Status_Divorced',
    'Income_Category_$60K - $80K',
    'Income_Category_Less than $40K',
    'Income_Category_$80K - $120K',
    'Income_Category_$40K - $60K',
    #'Income_Category_$120K +',
    'Card_Category_Gold',
    'Card_Category_Silver',
    'Card_Category_Platinum'
               ]]
X_3 = sm.add_constant(X_3)

In [52]:
log_reg_3 = sm.Logit(y_train_logreg.astype(float), X_3.astype(float)).fit()

Optimization terminated successfully.
         Current function value: 0.238549
         Iterations 8


In [53]:
print(log_reg_3.summary())

                           Logit Regression Results                           
Dep. Variable:         Attrition_Flag   No. Observations:                 5310
Model:                          Logit   Df Residuals:                     5287
Method:                           MLE   Df Model:                           22
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.4545
Time:                        16:16:48   Log-Likelihood:                -1266.7
converged:                       True   LL-Null:                       -2322.0
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -7.3161      0.615    -11.901      0.000      -8.521      -6.111
Customer_Age                       0.0190      0.006      2.977      0.

**BEZ Avg_Utilization_Ratio**

In [54]:
X_4 = X_train_logreg[[
    'Customer_Age',
    'Dependent_count',
    #'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    #'Credit_Limit', 
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    #'Avg_Utilization_Ratio',
    'Gender_M',
    #'Education_Level_High School',
    #'Education_Level_Graduate',
    #'Education_Level_Uneducated',
    #'Education_Level_College',
    #'Education_Level_Post-Graduate',
    #'Education_Level_Doctorate',
    'Marital_Status_Married',
    'Marital_Status_Single',
    #'Marital_Status_Divorced',
    'Income_Category_$60K - $80K',
    'Income_Category_Less than $40K',
    'Income_Category_$80K - $120K',
    'Income_Category_$40K - $60K',
    #'Income_Category_$120K +',
    'Card_Category_Gold',
    'Card_Category_Silver',
    'Card_Category_Platinum'
               ]]
X_4 = sm.add_constant(X_4)

/home/adam_s/um/moje_srod/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [55]:
log_reg_4 = sm.Logit(y_train_logreg.astype(float), X_4.astype(float)).fit()

Optimization terminated successfully.
         Current function value: 0.238666
         Iterations 8


In [56]:
print(log_reg_4.summary())

                           Logit Regression Results                           
Dep. Variable:         Attrition_Flag   No. Observations:                 5310
Model:                          Logit   Df Residuals:                     5288
Method:                           MLE   Df Model:                           21
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.4542
Time:                        16:17:58   Log-Likelihood:                -1267.3
converged:                       True   LL-Null:                       -2322.0
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -7.2940      0.614    -11.872      0.000      -8.498      -6.090
Customer_Age                       0.0190      0.006      2.974      0.

In [60]:
x_vars = log_reg_4.summary2().tables[1].index
#x_vars
wald_str = ' = '.join(list(x_vars[15:19]))
wald_str


'Income_Category_$60K - $80K = Income_Category_Less than $40K = Income_Category_$80K - $120K = Income_Category_$40K - $60K'

In [61]:
wald_test = log_reg_4.wald_test(wald_str) # joint test
wald_test

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[12.70740372]], p-value=0.005314046822330378, df_denom=3>

In [62]:
x_vars = log_reg_4.summary2().tables[1].index
#x_vars
wald_str = ' = '.join(list(x_vars[19:22]))
wald_str


'Card_Category_Gold = Card_Category_Silver = Card_Category_Platinum'

In [63]:
wald_test = log_reg_4.wald_test(wald_str) # joint test
wald_test

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[1.70146941]], p-value=0.42710102424866603, df_denom=2>

In [66]:
x_vars = log_reg_4.summary2().tables[1].index
#x_vars
wald_str = ' = '.join(list(x_vars[13:15]))
wald_str


'Marital_Status_Married = Marital_Status_Single'

In [68]:
wald_test = log_reg_4.wald_test(wald_str) # wald test
wald_test

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[31.64116951]], p-value=1.8545596792041255e-08, df_denom=1>

In [114]:
for col in X_4:
    print(X_4[col].unique())

[1.]
[46 51 45 44 48 40 53 61 43 37 39 56 34 50 58 49 36 41 35 54 52 47 42 57
 64 59 60 33 65 55 62 63 32 31 38 29 30 26 27 28 66 67 70]
[4 1 2 3 0 5]
[2 1 3 5 4 6]
[1 3 2 4 6 5 0]
[1 2 3 4 0 6 5]
[1043 2416    0 ... 2043 2248  479]
[ 1128. 24902.  4688. ... 24569.  1076.  6547.]
[0.7   0.917 0.593 0.658 0.807 0.324 0.949 1.466 0.548 0.718 0.522 0.744
 0.861 1.467 0.605 0.358 0.939 0.491 0.692 0.666 1.306 0.897 0.792 0.806
 0.796 1.07  0.699 0.681 0.411 0.747 0.906 0.727 0.802 0.753 0.665 0.628
 0.673 0.36  0.588 0.677 0.903 0.597 0.779 0.693 0.57  0.788 0.551 0.805
 0.661 0.713 2.121 0.896 0.734 0.708 0.812 0.196 0.716 0.75  0.241 0.781
 1.043 0.514 0.729 0.384 0.698 0.872 1.139 0.621 0.532 0.767 0.892 0.776
 0.943 0.848 0.959 1.047 0.675 0.787 0.749 0.682 0.864 0.791 0.371 0.967
 0.78  0.504 0.687 0.444 0.657 0.846 0.701 0.631 0.742 0.814 0.853 1.454
 0.578 0.794 0.583 1.046 1.028 1.214 0.67  0.579 0.573 0.418 0.875 0.816
 1.004 0.825 0.642 0.733 0.815 0.73  0.777 0.68  0.838 0.562 0

**Ilorazy szans**

In [109]:
np.exp(log_reg_4.params)

const                              0.000680
Customer_Age                       1.019188
Dependent_count                    0.882301
Total_Relationship_Count           1.523981
Months_Inactive_12_mon             0.614956
Contacts_Count_12_mon              0.616684
Total_Revolving_Bal                1.000967
Avg_Open_To_Buy                    1.000017
Total_Amt_Chng_Q4_Q1               1.688674
Total_Trans_Amt                    0.999510
Total_Trans_Ct                     1.125649
Total_Ct_Chng_Q4_Q1               14.330670
Gender_M                           2.628595
Marital_Status_Married             1.950075
Marital_Status_Single              1.036945
Income_Category_$60K - $80K        1.547514
Income_Category_Less than $40K     2.196245
Income_Category_$80K - $120K       1.112869
Income_Category_$40K - $60K        2.468056
Card_Category_Gold                 0.435325
Card_Category_Silver               0.643871
Card_Category_Platinum             0.173196
dtype: float64

In [75]:
X_4_test = X_test_logreg[[
    'Customer_Age',
    'Dependent_count',
    #'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    #'Credit_Limit', 
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    #'Avg_Utilization_Ratio',
    'Gender_M',
    #'Education_Level_High School',
    #'Education_Level_Graduate',
    #'Education_Level_Uneducated',
    #'Education_Level_College',
    #'Education_Level_Post-Graduate',
    #'Education_Level_Doctorate',
    'Marital_Status_Married',
    'Marital_Status_Single',
    #'Marital_Status_Divorced',
    'Income_Category_$60K - $80K',
    'Income_Category_Less than $40K',
    'Income_Category_$80K - $120K',
    'Income_Category_$40K - $60K',
    #'Income_Category_$120K +',
    'Card_Category_Gold',
    'Card_Category_Silver',
    'Card_Category_Platinum'
               ]]
X_4_test = sm.add_constant(X_4_test)

/home/adam_s/um/moje_srod/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [106]:
y_test_logreg = y_test_logreg.astype(float)
y_test_logreg.describe()

count    1771.000000
mean        0.846979
std         0.360109
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Attrition_Flag, dtype: float64

In [77]:
X_4

,const,Customer_Age,Dependent_count,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,...,Gender_M,Marital_Status_Married,Marital_Status_Single,Income_Category_$60K - $80K,Income_Category_Less than $40K,Income_Category_$80K - $120K,Income_Category_$40K - $60K,Card_Category_Gold,Card_Category_Silver,Card_Category_Platinum
8101,1.0,46,4,2,1,1,1043,1128.0,0.700,4531,...,1,1,0,0,0,1,0,0,0,0
8916,1.0,51,1,1,3,2,2416,24902.0,0.917,7481,...,1,0,1,0,0,1,0,0,0,0
7945,1.0,45,4,1,3,3,0,4688.0,0.593,5248,...,0,0,1,0,0,0,1,0,0,0
10061,1.0,44,2,3,2,3,2028,2154.0,0.658,16920,...,0,0,1,0,1,0,0,0,0,0
4743,1.0,48,3,3,2,4,0,19075.0,0.807,2013,...,1,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,1.0,49,3,5,3,4,0,1438.3,0.929,4205,...,0,1,0,0,1,0,0,0,0,0
7365,1.0,44,5,3,3,1,1726,983.0,0.698,5056,...,0,0,1,0,1,0,0,0,0,0
7413,1.0,50,1,4,3,2,0,2317.0,0.734,2214,...,1,0,1,1,0,0,0,0,0,0
7681,1.0,49,4,5,3,2,2133,905.0,0.689,4784,...,0,1,0,0,1,0,0,0,0,0


In [78]:
X_4_test

,const,Customer_Age,Dependent_count,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,...,Gender_M,Marital_Status_Married,Marital_Status_Single,Income_Category_$60K - $80K,Income_Category_Less than $40K,Income_Category_$80K - $120K,Income_Category_$40K - $60K,Card_Category_Gold,Card_Category_Silver,Card_Category_Platinum
9866,1.0,34,2,4,2,2,1442,6152.0,0.753,15026,...,0,0,1,0,0,0,1,0,0,0
9239,1.0,44,4,2,2,3,1727,7955.0,0.880,14991,...,1,0,1,1,0,0,0,0,0,0
963,1.0,48,2,5,1,3,2439,2131.0,0.693,1520,...,0,1,0,0,1,0,0,0,0,0
1999,1.0,26,0,3,2,3,1428,14.0,0.496,1134,...,0,0,1,0,1,0,0,0,0,0
2531,1.0,59,1,6,2,2,1965,335.0,0.596,1693,...,1,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9288,1.0,52,3,2,2,1,1518,7712.0,0.774,14082,...,0,1,0,0,1,0,0,0,0,0
1961,1.0,33,1,4,3,3,0,34516.0,0.636,2804,...,1,0,1,0,0,1,0,0,0,0
6059,1.0,48,2,4,2,2,0,4776.0,0.626,4581,...,1,1,0,0,0,0,1,0,0,0
9300,1.0,58,1,1,1,3,1341,33175.0,0.812,13988,...,1,1,0,0,0,1,0,0,0,0


In [100]:
predicted_y_logreg = log_reg_4.predict(X_4_test)
predicted_y_logreg = np.round(predicted_y_logreg, 0).astype(float)
predicted_y_logreg

9866    1.0
9239    1.0
963     1.0
1999    0.0
2531    1.0
       ... 
9288    1.0
1961    1.0
6059    1.0
9300    1.0
2827    1.0
Length: 1771, dtype: float64

In [104]:
print(metrics.accuracy_score(y_test_logreg, predicted_y_logreg))
print(metrics.confusion_matrix(y_test_logreg, predicted_y_logreg))
print(metrics.classification_report(y_test_logreg, predicted_y_logreg))

0.9045736871823828
[[ 157  114]
 [  55 1445]]
              precision    recall  f1-score   support

         0.0       0.74      0.58      0.65       271
         1.0       0.93      0.96      0.94      1500

    accuracy                           0.90      1771
   macro avg       0.83      0.77      0.80      1771
weighted avg       0.90      0.90      0.90      1771



In [110]:
np.exp(0.8761298)

2.4015870748708004

In [111]:
np.exp(-0.0166972)

0.983441425618327